## Notebook to calculate all important metrics for the networks and save them

## Imports

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import pypsa
import warnings

warnings.filterwarnings("ignore")


# imported own functions
from utils import market_values, capacity, generation
from utils import generation_storage_units, consumption_storage_units, capacity_storage_units, capacity_storage_units_con, market_values_storage_units, market_values_storage_units_con
from utils import generation_links, generation_links_bus, capacity_links, capacity_links_bus, market_values_links, market_values_links_bus

# imported own definitions
from utils import resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal, nodal_balance

# general variables
onshore_regions = gpd.read_file("../data/external/regions_onshore_elec_s_181.geojson")
offshore_regions = gpd.read_file("../data/external/regions_offshore_elec_s_181.geojson")
onshore_regions = onshore_regions.set_index('name')
offshore_regions = offshore_regions.set_index('name')

# Regions
onshore_regions['coords'] = onshore_regions['geometry'].apply(lambda x: x.representative_point().coords[:])
onshore_regions['coords'] = [coords[0] for coords in onshore_regions['coords']]
onshore_regions["name"] = onshore_regions.index
offshore_regions['coords'] = offshore_regions['geometry'].apply(lambda x: x.representative_point().coords[:])
offshore_regions['coords'] = [coords[0] for coords in offshore_regions['coords']]
offshore_regions["name"] = offshore_regions.index

# carrier lists
from utils import c_el_gen_s, c_el_con_s, c_h2_gen, c_h2_con

# Notebook Definitions
c1_groups = [resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal]
c1_groups_name = ["resistive heater", "gas boiler", "heat pump", "water tanks charger", "water tanks discharger",
                  "solar thermal"]

In [ ]:
# Network imports
stst = pypsa.Network("../data/raw/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc")
exp = pypsa.Network("../data/raw/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2030.nc")

## Spatial dfs

### old CALC

In [ ]:
# calc market values, generation, lmps, capacity factors for generators, links and storage units
for n in [stst, exp]:
    df_regions_onshore = onshore_regions.copy()
    df_regions_offshore = offshore_regions.copy()

    # function for carriers in n.generators.carrier.unique() # 13 / 13
    for carrier in n.generators.carrier.unique():
        df_regions_onshore[f"{carrier}_mv"] = market_values(n, carrier)
        df_regions_offshore[f"{carrier}_mv"] = market_values(n, carrier)
        df_regions_onshore[f"{carrier}_gen"] = generation(n, carrier) / 1000 * 3
        df_regions_offshore[f"{carrier}_gen"] = generation(n, carrier) / 1000 * 3
        df_regions_onshore[f"{carrier}_cap"] = capacity(n, carrier) / 1000
        df_regions_offshore[f"{carrier}_cap"] = capacity(n, carrier) / 1000
        # lmps
        # capacity factors This calculation is correct? as capacity is multiplied by 2920 is the same as multiplying the generation by 3 and then dividing it by the capacity times 8760 (as cap is in MW?)
        df_regions_onshore[f"{carrier}_cf"] = generation(n, carrier) / (capacity(n, carrier) * 2920)
        df_regions_offshore[f"{carrier}_cf"] = generation(n, carrier) / (capacity(n, carrier) * 2920)

    # function for carriers in n.links.carrier.unique() # 53 / 55
    for carrier in n.links.carrier.unique():
        df_regions_onshore[f"{carrier}_mv"] = market_values_links(n, carrier)
        df_regions_onshore[f"{carrier}_gen"] = generation_links(n, carrier) / 1000 * 3
        df_regions_onshore[f"{carrier}_cap"] = capacity_links(n, carrier) / 1000
        df_regions_onshore[f"{carrier}_cf"] = generation_links(n, carrier) / (capacity_links(n, carrier) * 2920)

    # function for carriers in n.storage_units.carrier.unique() # 2 / 2
    for carrier in n.storage_units.carrier.unique():
        df_regions_onshore[f"{carrier}_mv"] = market_values_storage_units(n, carrier)
        df_regions_onshore[f"{carrier}_gen"] = generation_storage_units(n, carrier) / 1000 * 3
        df_regions_onshore[f"{carrier}_cap"] = capacity_storage_units(n, carrier) / 1000
        # capacity factors (both generation and consumption(loading) is considered
        gen = abs(n.storage_units_t.p.loc[:, n.storage_units.carrier == carrier])
        gen.columns = gen.columns.map(n.storage_units.bus)
        gen.columns = gen.columns.map(n.buses.location)
        df_regions_onshore[f"{carrier}_cf"] = gen.sum() / (2*capacity_storage_units(n, carrier) * 2920)

    # set market values to nan where generation in corresponding region is lower than % quantile ( for generators, links, su)
    qt = 0.2
    for carrier in (n.generators.carrier.unique().tolist() +
                    n.links.carrier.unique().tolist() +
                    n.storage_units.carrier.unique().tolist()):
        # onshore
        index = df_regions_onshore[f"{carrier}_gen"] <= np.nanquantile(df_regions_onshore[f"{carrier}_gen"], qt)
        df_regions_onshore[f"{carrier}_mv_qt"] = df_regions_onshore[f"{carrier}_mv"]
        df_regions_onshore[f"{carrier}_mv_qt"][index] = np.nan

        # offshore
        if carrier in ["offwind-dc", "offwind-ac"]:
            index = df_regions_offshore[f"{carrier}_gen"] <= np.nanquantile(df_regions_offshore[f"{carrier}_gen"], qt)
            df_regions_offshore[f"{carrier}_mv_qt"] = df_regions_offshore[f"{carrier}_mv"]
            df_regions_offshore[f"{carrier}_mv_qt"][index] = np.nan

    # calc lmps at the buses (lmps that are only present for EU (e.g. oil) are nan at the moment)
    # TODO: decide if EU lmps are used as lmp for all regions
    for carrier_bus in n.buses.carrier.unique():
        # index would be same names as the bus (not the location as it is in the index of
        # df_regions_onshore so far -> map location to make sure the right lmp is set
        locs = n.buses.location[n.buses[n.buses.carrier == carrier_bus].index]
        # simple mean of lmps
        lmps = n.buses_t.marginal_price[n.buses[n.buses.carrier == carrier_bus].index].mean()
        df = pd.concat([lmps, locs], axis=1).rename(columns={0: f"{carrier_bus}_lmp"})
        df.set_index("location", inplace=True)
        if df.size == 1:
            if df.index == "EU":
                df = pd.DataFrame(np.repeat(df.values, 181), index=df_regions_onshore.index,
                                  columns=[f"{carrier_bus}_lmp"])
                df_regions_onshore[f"{carrier_bus}_lmp"] = df[f"{carrier_bus}_lmp"]
        else:
            df_regions_onshore[f"{carrier_bus}_lmp"] = df[f"{carrier_bus}_lmp"]

    if n == stst:
        df_stst_ons = df_regions_onshore
        df_stst_off = df_regions_offshore

    if n == exp:
        df_exp_ons = df_regions_onshore
        df_exp_off = df_regions_offshore

In [ ]:
# subdivide all carriers: stst.buses.carrier.unique().tolist()

electricity = ['AC', 'battery', 'Li ion', 'low voltage', 'home battery' ]
hydrogen = ['H2', 'H2 liquid']
heat = ['residential rural heat',
 'residential rural water tanks',
 'services rural heat',
 'services rural water tanks',
 'residential urban decentral heat',
 'residential urban decentral water tanks',
 'services urban decentral heat',
 'services urban decentral water tanks',
 'urban central heat',
 'urban central water tanks']
gas = ['gas', 'biogas', 'gas for industry']
oil = ['oil']
biomass = ['solid biomass','solid biomass for industry']
co2 = ['co2','co2 stored']
process_emisisons = ['process emissions']

c_tags = {
    'AC': "el",
    'battery': "el",
    'Li ion': "el",
    'low voltage': "el",
    'home battery': "el",
    'H2': "h2",
    'H2 liquid': "h2",
    'residential rural heat': "heat",
    'residential rural water tanks': "heat",
    'services rural heat': "heat",
    'services rural water tanks': "heat",
    'residential urban decentral heat': "heat",
    'residential urban decentral water tanks': "heat",
    'services urban decentral heat': "heat",
    'services urban decentral water tanks': "heat",
    'urban central heat': "heat",
    'urban central water tanks': "heat",
    'gas': "gas",
    'biogas': "gas",
    'gas for industry': "gas",
    'oil': "oil",
    'solid biomass': "biom",
    'solid biomass for industry': "biom",
    'co2': "co2",
    'co2 stored': "co2",
    'process emissions': "pe"
}

### new CALC

In [ ]:
# calc demand weighted lmps for selected buses

dw_lmp_buses = ["AC", "low voltage", "H2"]

for n, dfs in zip([stst, exp], [df_stst_ons,df_exp_ons]):
    for c in dw_lmp_buses:

        # demand weighted lmp per region
        nb = nodal_balance(n, carrier=c, time="2013", aggregate=['component'], energy=True)
        nb = nb.unstack(level=[1])
        weights = nb[nb < 0].groupby(by="snapshot").sum().abs()
        lmps = n.buses_t.marginal_price.loc[:, n.buses.carrier == c]
        dw_lmps = np.multiply(lmps, weights / weights.sum()).sum()
        dw_lmps.index = dw_lmps.index.map(n.buses.location)
        dfs[f"{c}_dw_lmp"] = dw_lmps

In [ ]:
# df_stst_ons, df_stst_off, df_exp_ons, df_exp_off
# conventions: consumption is negative, values in GW, GWh
# subdivide gen, mv, cap and cf into electricity, heat, hydrogen, .

buses = ["bus0", "bus1", "bus2", "bus3", "bus4"]
ps = ["p0", "p1", "p2", "p3", "p4"]

for n, dfs in zip([stst, exp], [[df_stst_ons, df_stst_off], [df_exp_ons, df_exp_off]]):

    # generators: can only generate energy, have only one bus, have only one capacity, can have only one carrier
    for c in n.generators.carrier.unique().tolist():
        # get tag
        c_bus = n.generators[n.generators.carrier == c].bus.map(n.buses.carrier).unique()[0]
        c_tag = c_tags[c_bus]
        for df in dfs:
            df[f"{c}_gen_{c_tag}"] = generation(n, c) / 1000 * 3
            df[f"{c}_cap_gen_{c_tag}"] = capacity(n, c) / 1000
            df[f"{c}_mv_gen_{c_tag}"] = market_values(n, c)
            df[f"{c}_cf_gen_{c_tag}"] = generation(n, c) / (capacity(n, c) * 2920)
            # only for onshore
            if len(df) != 100:
                df[f"{c}_mv-lmp_gen_{c_tag}"] = df[f"{c}_mv_gen_{c_tag}"] - df[f"{c_bus}_lmp"]
                df[f"{c}_vf_gen_{c_tag}"] = df[f"{c}_mv_gen_{c_tag}"] / df[f"{c_bus}_lmp"]
                # calc demand weighted value factor for selected buses
                if c_bus in dw_lmp_buses:
                    df[f"{c}_vf_dw_gen_{c_tag}"] = df[f"{c}_mv_gen_{c_tag}"] / df[f"{c_bus}_dw_lmp"]


    # storage units: can generate and consume, have only one bus, have twoe capacities?, can have only one carrier
    for c in n.storage_units.carrier.unique().tolist():
        # get tag
        c_bus = n.storage_units[n.storage_units.carrier == c].bus.map(n.buses.carrier).unique()[0]
        c_tag = c_tags[c_bus]
        for df in dfs:
            df[f"{c}_gen_{c_tag}"] = generation_storage_units(n, c) / 1000 * 3
            df[f"{c}_con_{c_tag}"] = consumption_storage_units(n, c) / 1000 * 3
            df[f"{c}_cap_gen_{c_tag}"] = capacity_storage_units(n, c) / 1000
            df[f"{c}_cap_con_{c_tag}"] = capacity_storage_units_con(n, c) / 1000
            df[f"{c}_mv_gen_{c_tag}"] = market_values_storage_units(n, c)
            df[f"{c}_mv_con_{c_tag}"] = market_values_storage_units_con(n, c)
            df[f"{c}_cf_gen_{c_tag}"] = generation_storage_units(n, c) / (capacity_storage_units(n, c) * 2920)
            df[f"{c}_cf_con_{c_tag}"] = consumption_storage_units(n, c) / (capacity_storage_units(n, c) * 2920)
            # capacity factor of consumption and generation
            df[f"{c}_cf_gen+con_{c_tag}"] = (generation_storage_units(n, c) + consumption_storage_units(n, c)) / ((capacity_storage_units(n, c) + capacity_storage_units_con(n, c)) * 2920)
            # only for onshore
            if len(df) != 100:
                df[f"{c}_mv-lmp_gen_{c_tag}"] = df[f"{c}_mv_gen_{c_tag}"] - df[f"{c_bus}_lmp"]
                df[f"{c}_vf_gen_{c_tag}"] = df[f"{c}_mv_gen_{c_tag}"] / df[f"{c_bus}_lmp"]
                df[f"{c}_mv-lmp_con_{c_tag}"] = df[f"{c}_mv_con_{c_tag}"] - df[f"{c_bus}_lmp"]
                df[f"{c}_vf_con_{c_tag}"] = df[f"{c}_mv_con_{c_tag}"] / df[f"{c_bus}_lmp"]
                # calc demand weighted value factor for selected buses
                if c_bus in dw_lmp_buses:
                    df[f"{c}_vf_dw_gen_{c_tag}"] = df[f"{c}_mv_gen_{c_tag}"] / df[f"{c_bus}_dw_lmp"]
                    df[f"{c}_vf_dw_con_{c_tag}"] = df[f"{c}_mv_con_{c_tag}"] / df[f"{c_bus}_dw_lmp"]

    # links: can generate and consume, have several buses with different numbers, have several capacities, can have several carriers even per bus (e.g. DAC bus3 generates heat for urban central and urban decentral)
    for c in n.links.carrier.unique():
        for i, bus in enumerate(buses):
            # check if bus exists
            if n.links[n.links.carrier == c][bus][0] != "":
                # tag
                c_bus = n.links[n.links.carrier == c][bus].map(n.buses.carrier).unique()[0]
                c_tag = c_tags[c_bus]

                for df in dfs:
                    # check if consumption or generation
                    gen = generation_links_bus(n, c, i)
                    gen_tag = "gen" if gen.sum() > 0 else "con"
                    df[f"{c}_{gen_tag}_{c_tag}"] = gen / 1000 * 3
                    df[f"{c}_cap_{gen_tag}_{c_tag}"] = capacity_links_bus(n, c, i) / 1000
                    df[f"{c}_cf_{gen_tag}_{c_tag}"] = abs(gen) / (capacity_links_bus(n, c, i) * 2920)
                    df[f"{c}_mv_{gen_tag}_{c_tag}"] = market_values_links_bus(n, c, i)
                    # only for onshore
                    if len(df) != 100:
                        df[f"{c}_mv-lmp_{gen_tag}_{c_tag}"] = df[f"{c}_mv_{gen_tag}_{c_tag}"] - df[f"{c_bus}_lmp"]
                        df[f"{c}_vf_{gen_tag}_{c_tag}"] = df[f"{c}_mv_{gen_tag}_{c_tag}"] / df[f"{c_bus}_lmp"]
                        # calc demand weighted value factor for selected buses
                        if c_bus in dw_lmp_buses:
                            df[f"{c}_vf_dw_{gen_tag}_{c_tag}"] = df[f"{c}_mv_{gen_tag}_{c_tag}"] / df[f"{c_bus}_dw_lmp"]


# set market values to nan where generation / consumption in corresponding region is lower / higher than 20%  / 80 % quantile
gen_qt = 0.2
con_qt = 0.8

for df in [df_stst_ons, df_stst_off, df_exp_ons, df_exp_off]:

    # generation
    for col in df.columns[df.columns.str.contains("mv_gen")].tolist():
        index = df[col.replace("mv_", "")] <= np.nanquantile(df[col.replace("mv_", "")], gen_qt)
        df[f"{col}_qt"] = df[col]
        df[f"{col}_qt"][index] = np.nan

    #consumption
    for col in df.columns[df.columns.str.contains("mv_con")].tolist():
        index = df[col.replace("mv_", "")] >= np.nanquantile(df[col.replace("mv_", "")], con_qt)
        df[f"{col}_qt"] = df[col]
        df[f"{col}_qt"][index] = np.nan

    # generation
    for col in df.columns[df.columns.str.contains("vf_gen")].tolist():
        index = df[col.replace("vf_", "")] <= np.nanquantile(df[col.replace("vf_", "")], gen_qt)
        df[f"{col}_qt"] = df[col]
        df[f"{col}_qt"][index] = np.nan

    #consumption
    for col in df.columns[df.columns.str.contains("vf_con")].tolist():
        index = df[col.replace("vf_", "")] >= np.nanquantile(df[col.replace("vf_", "")], con_qt)
        df[f"{col}_qt"] = df[col]
        df[f"{col}_qt"][index] = np.nan

    # generation
    for col in df.columns[df.columns.str.contains("mv-lmp_gen")].tolist():
        index = df[col.replace("mv-lmp_", "")] <= np.nanquantile(df[col.replace("mv-lmp_", "")], gen_qt)
        df[f"{col}_qt"] = df[col]
        df[f"{col}_qt"][index] = np.nan

    #consumption
    for col in df.columns[df.columns.str.contains("mv-lmp_con")].tolist():
        index = df[col.replace("mv-lmp_", "")] >= np.nanquantile(df[col.replace("mv-lmp_", "")], con_qt)
        df[f"{col}_qt"] = df[col]
        df[f"{col}_qt"][index] = np.nan

    # generation
    for col in df.columns[df.columns.str.contains("vf_dw_gen")].tolist():
        index = df[col.replace("vf_dw_", "")] <= np.nanquantile(df[col.replace("vf_dw_", "")], gen_qt)
        df[f"{col}_qt"] = df[col]
        df[f"{col}_qt"][index] = np.nan

    #consumption
    for col in df.columns[df.columns.str.contains("vf_dw_con")].tolist():
        index = df[col.replace("vf_dw_", "")] >= np.nanquantile(df[col.replace("vf_dw_", "")], con_qt)
        df[f"{col}_qt"] = df[col]
        df[f"{col}_qt"][index] = np.nan

### Consumed electric energy and prices payed

In [ ]:
# consumed electric energy and prices payed
# AC and low voltage are the main buses for electricity (why the difference?) Investigate in differences in lmps?

for n , df in zip([stst, exp], [df_stst_ons, df_exp_ons]):

    for c in c_el_con_s:

        # links
        if c in n.links.carrier.unique().tolist():
            # check if bus 0 is AC or low voltage bus
            if n.links[n.links.carrier == c].bus0.map(n.buses.carrier).unique() in ["AC", "low voltage"]:

                # consumption of link at bus 0
                con = n.links_t.p0.loc[:, n.links.carrier == c]
                con.columns = con.columns.map(n.links.bus0)
                # save consumption per location to df
                con_sum = con.sum()
                con_sum.index = con_sum.index.map(n.buses.location)
                # convert to TWh and make negative
                df[f"{c}_con_el2"] = con_sum / 1000 * 3 * -1
                # get lmp of buses where the links consumes from
                lmp_con = n.buses_t.marginal_price.loc[:, con.columns]
                # calculate costs for every time step and location
                overall_cost = con * lmp_con
                # calc consumption weighted average per location: overall cost per location / overall generation per location
                cost_mv = overall_cost.sum() / con.sum()
                cost_mv.index = cost_mv.index.map(n.buses.location)
                # save cost_mv to df (€/MWH_el)
                df[f"{c}_cost_mv_el"] = cost_mv

        # storage unit: only PHS can store
        elif c in n.storage_units.carrier.unique().tolist():

                # consumption of su
                con = n.storage_units_t.p_store.loc[:, n.storage_units.carrier == c]
                con.columns = con.columns.map(n.storage_units.bus)
                # save consumption per location to df in TWh and make negative
                df[f"{c}_con_el2"] = con.sum() / 1000 * 3 * -1
                # get lmp of buses where the su consumes from
                lmp_con = n.buses_t.marginal_price.loc[:, con.columns]
                # calculate costs for every time step and location
                overall_cost = con * lmp_con
                # calc consumption weighted average per location: overall cost per location / overall generation per location
                cost_mv = overall_cost.sum() / con.sum()
                # save cost_mv to df
                df[f"{c}_cost_mv_el"] = cost_mv

        else:
            print(f"{c} not found!")

        # set pries to nan where consumption in corresponding region is lower than % quantile
        # watch out for negative values here!!!!
        qt = 0.8
        index = df[f"{c}_con_el2"] >= np.nanquantile(df[f"{c}_con_el2"], qt)
        df[f"{c}_cost_mv_el_qt"] = df[f"{c}_cost_mv_el"]
        df[f"{c}_cost_mv_el_qt"][index] = np.nan

### Utilisation rate

In [ ]:
# utilisation rate

# what you actually want to measure is on how much of possible generation is utilized
# utilitazation rate???
# Problem if you calc the rate for every time step and location independently and then take the mean, all urs have the same weight. That makes no sense, as at times with almost no generation there is numerical issues with the rate
# better calc ur for every region as the ratio of the sum over all gen and the sum over all possible gen (make sure only valid regions make it to the plot, e.g. minimun generation of o,2 quantile)

th = 0.01 # MWh
method = 1

th_p = 0.01 # share of mean generation

# how much of the whole possible energy that can be generated is utilized (sum over all time steps and location than take ratio)
overall_ur_stst = pd.DataFrame(index=range(1))
overall_ur_exp = pd.DataFrame(index=range(1))

for n, df, overall_ur in zip([stst, exp], [df_stst_ons, df_exp_ons], [overall_ur_stst, overall_ur_exp]):

    # generators
    max_out_gen = n.generators_t.p_max_pu * n.generators.p_nom_opt[n.generators_t.p_max_pu.columns]
    real_out_gen = n.generators_t.p[n.generators_t.p_max_pu.columns]
    out_ratio_gen = (real_out_gen / max_out_gen) [real_out_gen > th]
    out_ratio_gen_sum = real_out_gen.sum() / max_out_gen.sum()

    # links
    # gen
    n_links_p1 = n.links_t.p1 *-1
    # some links have a static p_max_pu value and some have an alternating (series)
    index_series_li = n.links_t.p_max_pu.columns
    index_static_li = n.links.index.difference(n.links_t.p_max_pu.columns)

    # calculate the possible output for the link for every time step
    if method == 1:
        # use p_max_pu * p_nom_op
        max_output_links_static = n.links.loc[index_static_li].p_max_pu * n.links.loc[index_static_li].p_nom_opt
    elif method == 2:
        # alternatively use the maximum of the real output and set it as the maximum capacity
        max_output_links_static= n_links_p1[index_static_li].max()

    # make ts of max_output_links_static
    max_output_links_ts_static = n.links_t.p0[index_static_li].copy()
    for snap in n.links_t.p0.index:
        max_output_links_ts_static.loc[snap] = max_output_links_static[index_static_li]

    # calc time series of time dependent p_max_pu links
    max_output_links_ts_series = n.links_t.p_max_pu * n.links.p_nom_opt[index_series_li]

    # merge static and series values and reorder columns
    max_output_links_ts = pd.concat([max_output_links_ts_static, max_output_links_ts_series], axis=1)[n.links_t.p0.columns]

    # compare to real output
    out_ratio_links = (n_links_p1 / max_output_links_ts) [n_links_p1 > th]
    # calc out ratio weighted by generation (sum of all gen / sum of all cap / max_output)
    out_ratio_links_sum = n_links_p1.sum() / max_output_links_ts.sum()

    # storage units
    max_output = n.storage_units.p_max_pu * n.storage_units.p_nom_opt
    max_output_ts_su = n.storage_units_t.p.copy()
    for snap in n.storage_units_t.p.index:
        max_output_ts_su.loc[snap] = max_output[n.storage_units_t.p.columns]
    # compare to real output
    out_ratio_su = (n.storage_units_t.p_dispatch / max_output_ts_su) [n.storage_units_t.p_dispatch > th]
    out_ratio_su_sum = n.storage_units_t.p_dispatch.sum() / max_output_ts_su.sum()

    #######
    #######

    # gens
    for carrier in n.generators.carrier.unique():
        if carrier in ['gas', 'oil']:
            continue
        # calc ur as mean of all urs per time and space
        index = n.generators[n.generators.carrier == carrier].index
        ur = out_ratio_gen[index].mean()
        ur.index = ur.index.map(n.generators.bus).map(n.buses.location)
        df[f"{carrier}_ur_mean"] = ur

        # calc ur as ratio of sum of all gen and sum of all output
        ur_s = out_ratio_gen_sum[index][real_out_gen[index].sum() > real_out_gen[index].sum().mean() * th_p]
        ur_s.index = ur_s.index.map(n.generators.bus).map(n.buses.location)
        df[f"{carrier}_ur"] = ur_s

        # overall ur
        overall_ur[f"{carrier}"] = real_out_gen[index].sum().sum() / max_out_gen[index].sum().sum()

    # links
    for carrier in n.links.carrier.unique():
        # calc
        index = n.links[n.links.carrier == carrier].index
        ur = out_ratio_links[index].mean()
        ur.index = ur.index.map(n.links.bus1).map(n.buses.location)
        # group duplicate index entries
        ur = ur.groupby(by=["Link"], axis="index").mean()
        df[f"{carrier}_ur_mean"] = ur

        # calc ur as ratio of sum of all gen and sum of all output
        ur_s = out_ratio_links_sum[index][n_links_p1[index].sum() > n_links_p1[index].sum().mean() * th_p]
        ur_s.index = ur_s.index.map(n.links.bus1).map(n.buses.location)
        # group duplicate index entries
        ur_s = ur_s.groupby(by=["Link"], axis="index").mean()
        df[f"{carrier}_ur"] = ur_s

        # overall ur
        overall_ur[f"{carrier}"] = n_links_p1[index].sum().sum() / max_output_links_ts[index].sum().sum()

    # storage units
    for carrier in n.storage_units.carrier.unique():
        # calc
        index = n.storage_units[n.storage_units.carrier == carrier].index
        ur = out_ratio_su[index].mean()
        ur.index = ur.index.map(n.storage_units.bus).map(n.buses.location)
        df[f"{carrier}_ur_mean"] = ur

        #
        ur_s = out_ratio_su_sum[index][n.storage_units_t.p_dispatch[index].sum() > n.storage_units_t.p_dispatch[index].sum().mean() * th_p]
        ur_s.index = ur_s.index.map(n.storage_units.bus).map(n.buses.location)
        df[f"{carrier}_ur"] = ur_s

        # overall ur
        overall_ur[f"{carrier}"] = n.storage_units_t.p_dispatch[index].sum().sum() / max_output_ts_su[index].sum().sum()


In [ ]:
df_stst_ons.head()

In [ ]:
df_stst_off.head()

In [ ]:
df_exp_ons.head()

In [ ]:
df_exp_off.head()

In [ ]:
# save to pickle
df_stst_ons.to_pickle("../data/processed/df_stst_ons.pkl")
df_stst_off.to_pickle("../data/processed/df_stst_off.pkl")
df_exp_ons.to_pickle("../data/processed/df_exp_ons.pkl")
df_exp_off.to_pickle("../data/processed/df_exp_off.pkl")

overall_ur_stst.to_pickle("../data/processed/overall_ur_stst.pkl")
overall_ur_exp.to_pickle("../data/processed/overall_ur_exp.pkl")

In [ ]:
# verify reload
df_stst_ons = pd.read_pickle("../data/processed/df_stst_ons.pkl")
df_stst_off = pd.read_pickle("../data/processed/df_stst_off.pkl")
df_exp_ons = pd.read_pickle("../data/processed/df_exp_ons.pkl")
df_exp_off = pd.read_pickle("../data/processed/df_exp_off.pkl")

In [ ]:
df_stst_ons["Fischer-Tropsch_cap_con_h2"]

In [ ]:
df_stst_ons.head().dtypes

In [ ]:
df_stst_off.head()

In [ ]:
df_exp_ons.head()

In [ ]:
df_exp_off.head()

In [ ]:
all_c = exp.links.carrier.unique().tolist() + exp.generators.carrier.unique().tolist() + exp.storage_units.carrier.unique().tolist()

## Temporal dfs

In [ ]:
# calc electricity and hydrogen generation or consumption for every time step

df_stst_ts = pd.DataFrame(index=stst.generators_t.p.index)
df_exp_ts = pd.DataFrame(index=exp.generators_t.p.index)

for n, df in zip([stst, exp],[df_stst_ts, df_exp_ts]):
    for c in all_c: #(c_el_gen_s + c_el_con_s + c_h2_gen + c_h2_con):

        if c in n.generators.carrier.unique():
            c_bus = n.generators[n.generators.carrier == c].bus.map(n.buses.carrier).unique()[0]
            c_tag = c_tags[c_bus]
            df[f"{c}_gen_{c_tag}"] = n.generators_t.p.loc[:, n.generators.carrier == c].sum(axis=1) * 3

        elif c in n.links.carrier.unique():

            for i, bus in enumerate(buses):
                # check if bus exists
                if n.links[n.links.carrier == c][bus][0] != "":
                    # tag
                    c_bus = n.links[n.links.carrier == c][bus].map(n.buses.carrier).unique()[0]
                    c_tag = c_tags[c_bus]
                    # check if consumption or generation
                    gen = generation_links_bus(n, c, i)
                    gen_tag = "gen" if gen.sum() > 0 else "con"
                    df[f"{c}_{gen_tag}_{c_tag}"] = n.links_t[ps[i]].loc[:, n.links.carrier == c].sum(axis=1) * 3 * -1

        elif c in n.storage_units.carrier.unique():
            c_bus = n.storage_units[n.storage_units.carrier == c].bus.map(n.buses.carrier).unique()[0]
            c_tag = c_tags[c_bus]
            df[f"{c}_gen_el"] = n.storage_units_t.p_dispatch.loc[:, n.storage_units.carrier == c].sum(axis=1) * 3
            df[f"{c}_con_el"] = n.storage_units_t.p_store.loc[:, n.storage_units.carrier == c].sum(axis=1) * 3 * -1

        else:
            print(f"{c} is not a known carrier in {n}")

In [ ]:
df_stst_ts.columns[df_stst_ts.columns.str.contains("pipe")]

In [ ]:
df_stst_ts.head()

In [ ]:
# save to pickle
df_stst_ts.to_pickle("../data/processed/df_stst_ts.pkl")
df_exp_ts.to_pickle("../data/processed/df_exp_ts.pkl")

# verify reload
df_stst_ts = pd.read_pickle("../data/processed/df_stst_ts.pkl")
df_exp_ts = pd.read_pickle("../data/processed/df_exp_ts.pkl")

In [ ]:
df_stst_ts.head()

In [ ]:
df_exp_ts.head()